##  Introduction 


- The flora consists of about 2285 species.
- These species belong to 855 genera in 131 families.
- According to the present estimates, Saudi Arabia contains 97 trees (4.25%), 565 shrubs (24.73%) and about 1620 herbs (71.02%).
- Majority of species are seen in meadows, mountains and wadis (valleys) ( 70% flora).
- There are 110 families in the widely distributed plants.
- For this project I only collocted the data for the first 21 family .


Importing : 

In [ ]:
# Scraping imports :
from bs4 import BeautifulSoup
from urllib.request import urlopen, urlretrieve
from selenium import webdriver
import re
# Scraping Images :
import hashlib
from PIL import Image
import io, os
import requests
import time
from selenium import webdriver
# Saving csv files :
import glob
import os
import pandas as pd 
# Visualizing Data :
from PIL import Image
import matplotlib.pyplot as plt

### 1. Scraping the Names :
-  get the page data.
-  Store the data.
-  Fix the data.


This cell resulted in 1457 names. 

In [ ]:
# 1 : get the page data : 
from bs4 import BeautifulSoup
from urllib.request import urlopen, urlretrieve
from selenium import webdriver
import re
html_page = urlopen(
    r'http://plantdiversityofsaudiarabia.info/Biodiversity-Saudi-Arabia/Flora/Widely%20distributed/Widely%20Distributed%20species.htm'
)
soup = BeautifulSoup(html_page, "html.parser")
# 2 : Store the data : 
plants_names = []
for a in soup.find_all("i"):
    plants_names.append(a.get_text(strip=True))
len(plants_names)
## ================= Fixing the names :
# 3 : Fix the data : 
splants_names = str(plants_names)
plants_names = re.split('(?=[A-Z])', splants_names)
len(plants_names)


### 2. Scraping Images :
-  Function : get_image_urls > scroll_down , This function takes the query while scroll_down sub function makes sure the scraper is scrolling down the page. 
-  Function : s_state , This function report the state of the search and downloading. 
-  Function : search_download , This fuction send the search key to get_img_urls - it will download 20 image per key -and get reports from s_state function.

In [ ]:
def get_image_urls(query: str,
                     max_links_to_get: int,
                     wd: webdriver,
                     sleep_between_interactions: int = 1):
    '''

    '''
    def scroll_down(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)

    # add the query to google images url 
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_get:
        scroll_down(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)

        print(
            f"Here you go I've found : {number_results}  results. I'll start extracting the links from {results_start}:{number_results}"
        )

        for img in thumbnail_results[results_start:number_results]:
            
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception as e:
                print(e)
                continue

            # extract image urls
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute(
                        'src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_get:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls),
                  "image links, looking for more ...")
            time.sleep(30)
            #return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [ ]:
def s_state(folder_path: str, url: str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(
            folder_path,
            hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [ ]:
def search_download(search_term: str,
                        driver_path='chromedriver',
                        target_path='./images',
                        number_images=20):
    target_folder = os.path.join(target_path,
                                 '_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = get_image_urls(search_term,
                               number_images,
                               wd=wd,
                               sleep_between_interactions=0.5)

    for elem in res:
        s_state(target_folder, elem)


I've stopped the search at 225 , download images for 226 plant name which belong to 21 family. 

In [ ]:
# each time search for a plant name and download 20 images : 
for i in range(0, 225):
    word = plants_names[i] + " Plant"
    search_download(word, driver_path="chromedriver")

### 3. Preparing the Data : 
-  Reding  the images paths.
-  Reading the labels in the correct count and order.
- Saving the data.


In [ ]:
# reading the paths : 
my_path = r".\capStoneProject\images"
#print(my_path)
files=glob.glob(my_path + '/**/*.jpg', recursive=True)
#len(files) 4324


In [ ]:
# reaing labels :
labels = [
    "ACANTHACEAE", "AIZOACEAE", "ALLIACEAE", "ALOACEAE", "AMARANTHACEAE",
    "AMARYLLIDACEAE", "ANACARDIACEAE", "APOCYNACEAE", "ARISTOLOCHIACEAE",
    "ASCLEPIADACEAE", "ASPARAGACEAE", "ASPHODELACEAE", "BARBEYACEAE",
    "BERBERIDACEAE", "BORAGINACEAE", "BURSERACEAF", "CACTACEAE",
    "CAMPANULACEAE", "CAPPARACEAE", "CAPRIFOLIACEAE", "CARYOPHYLLACEAE"
]
l=[]
for i in labels:
    my_path = r"C:\Users\un_cs\Documents\capStoneProject\images"+r"\\"+i
    fil=glob.glob(my_path + '/**/*.jpg', recursive=True)
    for j in range(0,len(fil)):
        l.append(i)
len(l) # 4324

In [ ]:
# saving the data in csv file :
data=pd.DataFrame(
    {'label': l,
     'img': files,
    })
data.head()
########### saving data 
data.to_csv(r".\capStoneProject\data\data.csv", index =None)

### 4. Visualizing :

In [ ]:
# first 5 images :
size = (100, 100)
for image_path in data.iloc[:5,1]:
    im = Image.open(image_path)
    im = im.resize(size)
    plt.imshow(im)
    plt.show()
